# Systematics impact: Ranking Plot method

*Michele Pinamonti - INFN Sezione di Trieste*

This part shows an example of a way to quantify the impact of each of the individual sources of systematic uncertainties, using the so-called "ranking plot" method.
Each of the nuisance parameters is shifted up and down by its uncertainty, and then the fit is repeated.
Then the impact of associated systematic is quoted as the resulting shift of the result of the fit (in terms of fitted value of the parameter of interest).

In [ ]:
using namespace RooStats;

Open file containing workspace and extract it

In [ ]:
TFile *f = new TFile("../ws/ATLASIT_prova_combined_ATLASIT_prova_model.root");
RooWorkspace *w = (RooWorkspace*)f->Get("combined");

Get observed data from the workspace

In [ ]:
RooDataSet *dataset = (RooDataSet*)w->data("obsData");

Perform the nominal S+B fit to data (but forcing RooFit and Minuit to stay silent first)

In [ ]:
RooMsgService::instance().setGlobalKillBelow(RooFit::FATAL);
w->var("mu_ttH")->setVal(0);
w->var("mu_ttH")->setConstant(false);
w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));

Get the ModelConfig, the POI and save the POI fitted value and uncertainty

In [ ]:
ModelConfig *mc = (ModelConfig*)w->obj("ModelConfig");
RooRealVar *poi = (RooRealVar*)mc->GetParametersOfInterest()->first();
float mu_hat = poi->getVal();
float mu_hat_err_up = poi->getErrorHi();
float mu_hat_err_down = poi->getErrorLo();

Print nominal fit result

In [ ]:
cout << "Nominal fit:" << endl;
cout << "  POI =";
cout << Form(" %+.3f",mu_hat);
cout << Form(" %+.3f",mu_hat_err_up);
cout << " / ";
cout << Form(" %+.3f",mu_hat_err_down);
cout << endl;

Save a "snapshot", containing parameter values after nominal fit (to be loaded afeterward)

In [ ]:
w->saveSnapshot("nominal_snapshot", *mc->GetPdf()->getParameters(dataset));

Now the main part:
- loop on all the nuisance parmeters (NPs) in the model (excluding the free-floating parameters, hence restricting to those named `alpha_*`
- save value and error (up/down) of each NP from the nominal fit
- for each NP, repeat the fit after fixing the NP to nominal value +/- its post-fit uncertainty (to get the "post-fit impact of this NP")
- for each NP, repeat the fit after fixing the NP to nominal value +/- 1 (to get the "pre-fit impact of this NP")
- to get the "impact", compare fitted values of POI with nominal fit result in each case
- take care of loading the snapshot we created before, prior to any new fit
- print everything in an ordered way

In [ ]:
// loop on list of nuisance parameters
for(auto np_tmp : *mc->GetNuisanceParameters()){
    RooRealVar* np = (RooRealVar*)np_tmp;
    
    string np_name = np->GetName();
    
    // exclude parameters that are not "alpha_" (to exclude e.g. normalization factors and keep only NPs associated with systematics)
    if(np_name.find("alpha_")==string::npos) continue;
    
    cout << "---------------------------------" << endl;
    cout << "Impact of NP " << np_name << endl;
    
    // load the snapshot we created before, and save value and error (up/down) of this NP
    w->loadSnapshot("nominal_snapshot");
    float np_value = np->getVal();
    float np_err_up = np->getErrorHi();
    float np_err_down = np->getErrorLo();
    
    // Evaluate post-fit impact:
    //
    // set this NP to +1 sigma
    w->loadSnapshot("nominal_snapshot");
    np->setVal( np_value + np_err_up );
    np->setConstant(true);
    w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
    float mu_hat_postfit_up = poi->getVal();
    //
    // set this NP to -1 sigma
    w->loadSnapshot("nominal_snapshot");
    np->setVal( np_value + np_err_down );
    np->setConstant(true);
    w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
    float mu_hat_postfit_down = poi->getVal();
    
    // Evaluate pre-fit impact:
    //
    // set this NP to +1
    w->loadSnapshot("nominal_snapshot");
    np->setVal( np_value + 1 );
    np->setConstant(true);
    w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
    float mu_hat_prefit_up = poi->getVal();
    //
    // set this NP to -1
    w->loadSnapshot("nominal_snapshot");
    np->setVal( np_value - 1 );
    np->setConstant(true);
    w->pdf("simPdf")->fitTo(*dataset,RooFit::PrintLevel(-1));
    float mu_hat_prefit_down = poi->getVal();
    
    // Print pre-fit impact
    cout << "  Pre-fit  impact = ";
    cout << Form("%+.3f",mu_hat_prefit_up - mu_hat);
    cout << " / ";
    cout << Form("%+.3f",mu_hat_prefit_down - mu_hat);
    cout << endl;
    
    // Print post-fit impact
    cout << "  Post-fit impact = ";
    cout << Form("%+.3f",mu_hat_postfit_up - mu_hat);
    cout << " / ";
    cout << Form("%+.3f",mu_hat_postfit_down - mu_hat);
    cout << endl;
}